 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 3: Optimisation sans contraintes et méthodes itératives
Tangi Migot et Paul Raynaud

In [2]:
using LinearAlgebra, NLPModels, Printf

In [3]:
#Test problem:
using ADNLPModels
fH(x) = (x[2]+x[1].^2-11).^2+(x[1]+x[2].^2-7).^2
x0H = [10., 20.]
himmelblau = ADNLPModel(fH, x0H)

problem2 = ADNLPModel(x->-x[1]^2, ones(3))

roz(x) = 100 *  (x[2] - x[1]^2)^2 + (x[1] - 1.0)^2
rosenbrock = ADNLPModel(roz, [-1.2, 1.0])

f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
pb_du_cours = ADNLPModel(f, [-1.001, -1.001]) #ou [1.5, .5] ou [.5, .5]

ADNLPModel - Model with automatic differentiation backend ADModelBackend{
  ForwardDiffADGradient,
  ForwardDiffADHvprod,
  EmptyADbackend,
  EmptyADbackend,
  EmptyADbackend,
  ForwardDiffADHessian,
  EmptyADbackend,
}
  Problem name: Generic
   All variables: ████████████████████ 2      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   3               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                            

## Commentaires sur Julia

Quelques commentaires sur des morceaux de codes que vous avez vu:
- les structures, exemple [GenericExecutionStats](https://github.com/JuliaSmoothOptimizers/SolverCore.jl/blob/0091f437a26a27ac8aa53d5e37647223722f7f7c/src/stats.jl#L60) (constructeur, attribut, type).


In [5]:
using SolverCore

In [7]:
? GenericExecutionStats

ErrorException: syntax: invalid identifier name "?"

- Les arguments dans les fonctions. Lire attentivement [la documentation Julia sur les fonctions](https://docs.julialang.org/en/v1/manual/functions/) pour comprendre l'utilisation des `Optional Arguments` et des `Keywords Arguments`. Ce type d'arguments est très utile dans nos applictions où les solveurs dépendent de paramètre dont on peut fixer des valeurs par défaut.

## Exercice 1: Méthode BFGS avec mémoire limitée (L-BFGS)

Le but de cet exercice est d'implémenter la méthode BFGS à mémoire limitée vue en cours en utilisant les `InverseLBFGSOperator` du package `LinearOperators.jl`. Il y a aussi un petit exemple dans la documentation du package [LinearOperators.jl/dev/tutorial/#Limited-memory-BFGS-and-SR1](https://juliasmoothoptimizers.github.io/LinearOperators.jl/dev/tutorial/#Limited-memory-BFGS-and-SR1).

In [9]:
using LinearOperators

In [ ]:
? InverseLBFGSOperator

ce qui est important dans ce type de méthode est:
- le paramètre mémoire
- la mise à jour de l'opérateur avec la fonction `push!`
- si on a pas une direction de descente, alors on skip
- recherche linéaire d'Armijo

In [ ]:
? LinearOperators.push!

In [10]:
function armijo(xk, dk, fk, gk, slope, nlp :: AbstractNLPModel; τ1 = 1.0e-4, t_update = 1.5)
  t = 1.0
  fk_new = obj(nlp, xk + dk) # t = 1.0
  while fk_new > fk + τ1 * t * slope
    t /= t_update
    fk_new = obj(nlp, xk + t * dk)
  end
  return t, fk_new
end

armijo (generic function with 1 method)

In [11]:
function limited_bfgs(nlp      :: AbstractNLPModel;
                      x        :: AbstractVector = nlp.meta.x0,
                      atol     :: Real = √eps(eltype(x)), 
                      rtol     :: Real = √eps(eltype(x)),
                      max_eval :: Int = -1,
                      max_time :: Float64 = 30.0,
                      f_min    :: Float64 = -1.0e16,
                      verbose  :: Bool = true,
                      mem      :: Int = 5)
  start_time = time()
  elapsed_time = 0.0

  T = eltype(x)
  n = nlp.meta.nvar

  xt = zeros(T, n)
  ∇ft = zeros(T, n)

  f = obj(nlp, x)
  ∇f = grad(nlp, x)
#################################################
  H = InverseLBFGSOperator(T,n)
#################################################

  ∇fNorm = norm(∇f) #nrm2(n, ∇f)
  ϵ = atol + rtol * ∇fNorm
  iter = 0

  @info log_header([:iter, :f, :dual, :slope, :bk], [Int, T, T, T, T],
                   hdr_override=Dict(:f=>"f(x)", :dual=>"‖∇f‖", :slope=>"∇fᵀd"))

  optimal = ∇fNorm ≤ ϵ
  unbdd = f ≤ f_min
  tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  stalled = false
  status = :unknown

  while !(optimal || tired || stalled || unbdd)

#################################################
    d = -H*∇f
#################################################
    slope = dot(d, ∇f)
    if slope ≥ 0
      @error "not a descent direction" slope
      status = :not_desc
      stalled = true
      continue
    end

    # Perform improved Armijo linesearch.
    t, ft = armijo(x, d, f, ∇f, slope, nlp)
        
    @info log_row(Any[iter, f, ∇fNorm, slope, t])

    # Update L-BFGS approximation.
    xt = x + t * d
    ∇ft = grad(nlp, xt) # grad!(nlp, xt, ∇ft)
#################################################
    push!(H,xt-x,∇ft-∇f)
#################################################

    # Move on.
    x = xt
    f = ft
    ∇f = ∇ft

    ∇fNorm = norm(∇f) #nrm2(n, ∇f)
    iter = iter + 1

    optimal = ∇fNorm ≤ ϵ
    unbdd = f ≤ f_min
    elapsed_time = time() - start_time
    tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  end
  @info log_row(Any[iter, f, ∇fNorm])

  if optimal
    status = :first_order
  elseif tired
    if neval_obj(nlp) > max_eval ≥ 0
      status = :max_eval
    elseif elapsed_time > max_time
      status = :max_time
    end
  elseif unbdd
        status = :unbounded
  end

  return GenericExecutionStats(
        nlp,
        status=status,
        solution=x,
        objective=f,
        dual_feas=∇fNorm,
        iter=iter,
        elapsed_time=elapsed_time,
    )
end

limited_bfgs (generic function with 1 method)

In [13]:
#Unit/Validation Tests
# Réaliser un test unitaire

In [38]:
include("test-lbfgs")

(stats.status, stats.solution) = (:first_order, [3.584428266659278, -1.8481265666485829])
(stats.status, stats.solution) = 

(:unbounded, [1.29140163e8, 1.0, 1.0])


(stats.status, stats.solution) = (:first_order, [0.9999999887950609, 0.9999999782159007])
(stats.status, stats.solution) = 

(:unbounded, [-975544.6831042847, -764227.9248199855])
(stats.status, stats.solution) = (:first_order, [0.9999999962625671, -3.1168150200102845e-9])
(stats.status, stats.solution) = (:first_order, [0.99999999073849, -6.617373493448244e-9])


Test Summary: | Pass  Total  Time
test set      |    9      9  3.4s


Test.DefaultTestSet("test set", Any[], 9, false, false, true, 1.707162119081548e9, 1.70716212243425e9, false)

### Bonus:

- Compare l'implémentation de `limited_bfgs` avec la fonction `lbfgs` qui est disponible dans `JSOSolvers.jl`.
- On veut pouvoir tester "facilement" plusieurs valeurs de $\tau$ et du paramètre de mise à jour dans `armijo` sur les problèmes tests. Comment modifier le code pour que ça soit possible?

On peut mesurer deux executions de fonctions Julia grâce aux fonctions de `BenchmarkTools.jl`:

In [40]:
using BenchmarkTools
using JSOSolvers

In [ ]:
? @time

In [43]:
@time "lbfgs(himmelblau) :" lbfgs(himmelblau)
@time "limited_bfgs(himmelblau) :" limited_bfgs(himmelblau)

lbfgs(himmelblau) :: 0.000113 seconds (269 allocations: 10.773 KiB)


limited_bfgs(himmelblau) :: 0.015503 seconds (1.82 k allocations: 171.609 KiB)


┌ Info:   iter      f(x)      ‖∇f‖      ∇fᵀd        bk  
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:29
┌ Info:      0   1.7e+05   3.3e+04  -1.1e+09   1.0e-03
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      1   2.7e+04   8.6e+03  -5.9e+04   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      2   1.3e+03   8.6e+02  -6.6e+02   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      3   7.2e+02   5.7e+02  -8.8e+02   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      4   1.6e+02   2.0e+02  -1.6e+02   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      5   4.6e+01   8.6e+01  -5.4e+01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      6   1.0e+01 

"Execution stats: first-order stationary"

## Exercice 2: NewtonCG

Le but de cet exercice est d'adapter les méthodes de Newton de façon à résoudre le système linéaire avec une méthode itérative de type gradient conjugué comme suit ($B_k$ représente la matrice hessienne):

<img src="LineSearchNewtonCG.png" width=600 height=600 />

In [39]:
function cg_optim(H, ∇f)
    #setup the tolerance:
    n∇f = norm(∇f)
#####################################
    ϵk = min(0.5, √n∇f)*n∇f
####################################
    n = length(∇f)
    z = zeros(n)
    r = ∇f
    d = -r
    
    j = 0
    while norm(r) ≥ ϵk && j < 3 * n
###############################################
        if dot(d, H * d) ≤ 0
            if j == 0
                p = d
            else
                p = z 
            end
        end
##############################################
        α = dot(r,r)/dot(d,H*d)
##############################################        
        z += α * d
        nrr2 = dot(r, r)
        r += α * H * d
##############################################
        β  = dot(r, r)/nrr2
##############################################
        d  = -r + β * d
        j += 1
    end
    return z
end

cg_optim (generic function with 1 method)

Ce qui est important ici est qu'on a pas besoin de stocker/évaluer la matrice hessienne entière mais simplement le produit entre la hessienne et un vecteur. Pour un `NLPModels` on utilise:

In [44]:
using NLPModels

In [ ]:
? NLPModels.hprod

In [ ]:
? NLPModels.hess_op

In [ ]:
function armijo_Newton_cg(nlp      :: AbstractNLPModel;
                          x        :: AbstractVector = nlp.meta.x0,
                          atol     :: Real = √eps(eltype(x)), 
                          rtol     :: Real = √eps(eltype(x)),
                          max_eval :: Int = -1,
                          max_time :: Float64 = 30.0,
                          f_min    :: Float64 = -1.0e16)
  start_time = time()
  elapsed_time = 0.0

  T = eltype(x)
  n = nlp.meta.nvar

  f = obj(nlp, x)
  ∇f = grad(nlp, x)
#################################################
  H = # Initialize H as linear operator representing the Hessian matrix
#################################################

  ∇fNorm = norm(∇f) #nrm2(n, ∇f)
  ϵ = atol + rtol * ∇fNorm
  iter = 0

  @info log_header([:iter, :f, :dual, :slope, :bk], [Int, T, T, T, T],
                   hdr_override=Dict(:f=>"f(x)", :dual=>"‖∇f‖", :slope=>"∇fᵀd"))

  optimal = ∇fNorm ≤ ϵ
  unbdd = f ≤ f_min
  tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  stalled = false
  status = :unknown

  while !(optimal || tired || stalled || unbdd)
        
    d = cg_optim(H, ∇f)
        
    slope = dot(d, ∇f)
    if slope ≥ 0
      @error "not a descent direction" slope
      status = :not_desc
      stalled = true
      continue
    end

    # Perform improved Armijo linesearch.
    t, f = armijo(x, d, f, ∇f, slope, nlp)
        
    @info log_row(Any[iter, f, ∇fNorm, slope, t])

    # Update L-BFGS approximation.
    x += t * d
    ∇f = grad(nlp, x)
#################################################
    H = ### Update H
#################################################

    ∇fNorm = norm(∇f) #nrm2(n, ∇f)
    iter = iter + 1

    optimal = ∇fNorm ≤ ϵ
    unbdd = f ≤ f_min
    elapsed_time = time() - start_time
    tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  end
  @info log_row(Any[iter, f, ∇fNorm])

  if optimal
    status = :first_order
  elseif tired
    if neval_obj(nlp) > max_eval ≥ 0
      status = :max_eval
    elseif elapsed_time > max_time
      status = :max_time
    end
  elseif unbdd
        status = :unbounded
  end

  return GenericExecutionStats(nlp, status = status, solution=x, objective=f, dual_feas=∇fNorm,
                               iter=iter, elapsed_time=elapsed_time)
end

In [ ]:
#Unit/Validation Tests
# Réaliser un test unitaire

## Comment préparer un benchmark

On veut maintenant pouvoir réaliser un benchmark de plusieurs solveurs. Pour comparer les algorithmes, il nous faut une collection de problèmes tests et on va utiliser `OptimizationProblems.jl`.

In [ ]:
using OptimizationProblems

Vous pouvez trouver un tutoriel de comment réaliser un benchmark avec ce package sur la documentation [OptimizationProblems.jl/dev/benchmark/](https://juliasmoothoptimizers.github.io/OptimizationProblems.jl/dev/benchmark/).

Il est fort possible que les petits problèmes tests que l'on résout après l'implémentation ne suffisent pas à déceler des bugs. Mais on peut toujours analyser l'éxecution de notre algorithme sur certains problèmes de la collection afin d'améliorer la valeur de certains paramètres (limite de temps, d'itérations, d'évaluations), détecter un bug, etc.

In [ ]:
using OptimizationProblems.PureJuMP, NLPModelsJuMP
jump_model = AMPGO02() # OptimizationProblems.PureJuMP.AMPGO02
prbl = MathOptNLPModel(jump_model)
limited_bfgs(prbl)

Vous vous en doutez pour le rapport de cette semaine on va vouloir réaliser une benchmark avec les deux méthodes que l'on a codé.

### Appendix:

Une petite remarque sur la gestion de la mémoire:

In [3]:
#Pour les nombres:
a = 1
@show a
b = a
@show b
a = 2
@show b

a = 1
b = 1
b = 1


1

In [ ]:
#Pour les tableaux:
a = zeros(Float64, 2) #or zeros(2)
@show a
b = a
@show (a,b)
a = ones(Float64, 2)
@show (a,b)

#Pour les tableaux:
a = ones(Float64, 2)
b = a
@show (a,b)
a .= 2*ones(Float64, 2) #same would go with grad!
@show (a,b)

#Pour les tableaux:
a = ones(Float64, 2)
b = copy(a) # or similar(a)
@show (a,b)
a .= 2 .* ones(Float64, 2) #same would go with grad!
@show (a,b)

In [ ]:
#Pour les NLPModels, il existe aussi des fonctions qui interviennent sur la mémoire
gk = grad(nlp, x0)
grad!(nlp, x0, gk) #équivaut à gk .= grad(nlp, x0)

Comparez les `grad` et `grad!` à l'aide de `@benchmark` ainsi que:


In [ ]:
n = 10000
a = rand(n)
b = ones(n)
c = similar(a)
@benchmark c = 2 * a + b * 1.2


et

In [ ]:
@benchmark c .= 2 .* a .+ b .* 1.2

En particulier, observez la mémoire allouée et le temps moyen requis pour performer la ligne d'instruction.
La mémoire nécessaire varie en fonction du nombre et du type d'opération effectué.
Parmis les opérations allouant le moins de mémoire, on retrouve:

In [ ]:
@benchmark c .= 0

Pour manipuler plus finement la mémoire au cours de votre implémentation, vous pouvez utilisez des fonctions de `LinearAlgebra`

In [ ]:
? axpy!
? axpby!

qui sont des routines `BLAS`.
Faites attention, ces routines ont des effets de bord sur les structures de données `y`.
Dès lors, l'utilisation de `@benchmark` accumule successivement les effets de bord, d'où l'utilisation des tests avant `@benchmark`.

In [ ]:
y = zeros(n)
x = ones(n)
a = 2
axpy!(a, x, y)
@show y == a * x

@benchmark axpy!(a, x, y)

In [ ]:
y = π * ones(n)
x = ones(n)
a = 2
b = 3
axpby!(a, x, b, y)
@show y == a * ones(n) + b * π * ones(n)

@benchmark axpby!(a, x, b, y)

Cela fontionne aussi pour les matrices

In [ ]:
y = π * ones(n,n)
x = ones(n,n)
a = 2
b = 3
axpby!(a, x, b, y)
@show y == a * ones(n,n) + b * π * ones(n,n)

@benchmark axpby!(a, x, b, y)